In [2]:
cd ..

/home/scott/Documents/git/bite-me


In [3]:
import sklearn
import pandas as pd
import numpy as np
import datetime

In [4]:
from IPython.display import HTML
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

/usr/local/lib/python3.4/dist-packages/matplotlib/backends/backend_gtk3agg.py:18: UserWarning: The Gtk3Agg backend is known to not work on Python 3.x with pycairo. Try installing cairocffi.
  "The Gtk3Agg backend is known to not work on Python 3.x with pycairo. "


In [5]:
import sklearn.linear_model
import sklearn.cross_validation

In [6]:
from eat_it import StratifiedPercentileKFold

In [7]:
from eat_it import scalers

In [60]:
from eat_it import params

In [8]:
rng = sklearn.utils.check_random_state(0)

In [9]:
train = pd.read_csv('data/train.csv', encoding="utf-8")
# Add age in days
end_dt = datetime.datetime.strptime('2015-1-1', "%Y-%m-%d")
train['Age'] = [(end_dt - datetime.datetime.strptime(open_dt, "%m/%d/%Y")).days for open_dt in train['Open Date']]
# add size as boolean
train['isBig'] = train['City Group']=='Big Cities'
# add boolean for type
train['isIL'] = train['Type']=='IL'

In [10]:
gtest = pd.read_csv('data/genuinetest.csv', encoding="utf-8")
# Add age in days
end_dt = datetime.datetime.strptime('2015-1-1', "%Y-%m-%d")
gtest['Age'] = [(end_dt - datetime.datetime.strptime(open_dt, "%m/%d/%Y")).days for open_dt in gtest['Open Date']]
# add size as boolean
gtest['isBig'] = gtest['City Group']=='Big Cities'
# add boolean for type
gtest['isIL'] = gtest['Type']=='IL'

In [12]:
# add known revenues from public test data
gtestrevenue = pd.read_csv('data/genuinetestrevenue.csv', encoding="utf-8")

In [20]:
labelled_test = pd.merge(gtest, gtestrevenue, on='Id')

In [44]:
# Merge all available training data together
data = pd.concat((train, labelled_test), ignore_index=True)

In [45]:
data.shape

(167, 46)

In [46]:
# Remove DT type from the list
data[data['Type']=='DT']

,Age,City,City Group,Id,Open Date,P1,P10,P11,P12,P13,...,P4,P5,P6,P7,P8,P9,Type,isBig,isIL,revenue
124,3423,Ankara,Big Cities,124,08/18/2005,1,5,2,5,5,...,5,5,5,1,5,5,DT,True,False,3810007


In [47]:
# Remove DT type from the list
data = data[data['Type']!='DT']

In [48]:
data.shape

(166, 46)

In [49]:
# List all the columns we can train on
cols = ['P'+str(i) for i in range(1,38)]
cols.append('Age')
cols.append('isBig')
cols.append('isIL')

In [50]:
cols

['P1',
 'P2',
 'P3',
 'P4',
 'P5',
 'P6',
 'P7',
 'P8',
 'P9',
 'P10',
 'P11',
 'P12',
 'P13',
 'P14',
 'P15',
 'P16',
 'P17',
 'P18',
 'P19',
 'P20',
 'P21',
 'P22',
 'P23',
 'P24',
 'P25',
 'P26',
 'P27',
 'P28',
 'P29',
 'P30',
 'P31',
 'P32',
 'P33',
 'P34',
 'P35',
 'P36',
 'P37',
 'Age',
 'isBig',
 'isIL']

In [51]:
X = data.as_matrix(cols)

In [52]:
X = X.astype(np.float)

In [53]:
X.shape

(166, 40)

In [54]:
y = data['revenue'].values

In [55]:
X_indices = data['Id'].values

In [56]:
def do_cv(cv, clf, y_transform=None, X=X, y=y):
    if y_transform is None:
        y_transform = lambda x: x
    scores = []
    all_pred = np.zeros((y.shape))
    for train_index, test_index in cv:
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        # Fit model
        clf.fit(X_train, y_train)
        # Predict scores for test data
        y_pred = y_transform(clf.predict(X_test))
        # Save all the predictions to an array
        all_pred[test_index] = y_pred
        # Compute mean squared error on this test set
        mse = sklearn.metrics.mean_squared_error(y_transform(y_test),y_pred)
        scores.append(mse)
    # Compute MSE across all samples
    all_score = sklearn.metrics.mean_squared_error(y_transform(y), all_pred)**0.5
    scores = np.asarray(scores)**0.5
    return all_score, scores

In [57]:
clf = sklearn.linear_model.LinearRegression()

In [58]:
def get_cv_results(X=X, y=y, clf=clf, n_folds=10, n_samp=25):
    all_scores = []
    for seed in range(n_samp):
        cv = StratifiedPercentileKFold.StratifiedPercentileKFold(y, n_folds=n_folds, shuffle=True, random_state=seed, shuffle_windows=True)
        this_score, _ = do_cv(cv, clf, X=X, y=y)
        all_scores.append(this_score)
    return np.mean(all_scores), np.std(all_scores)/np.sqrt(n_folds)

def get_mean_cv_score(*args, **kwargs):
    out = get_cv_results(*args, **kwargs)
    return out[0]

In [62]:
params.xor_cols

['P14',
 'P15',
 'P16',
 'P17',
 'P18',
 'P24',
 'P25',
 'P26',
 'P27',
 'P30',
 'P31',
 'P32',
 'P33',
 'P34',
 'P35',
 'P36',
 'P37']

In [82]:
'P1' in params.xor_cols

False

In [70]:
data['missingSource'] = data[params.xor_cols].apply(lambda x: np.all(x==0), axis=1)

In [75]:
HTML(data.to_html())

,Age,City,City Group,Id,Open Date,P1,P10,P11,P12,P13,P14,P15,P16,P17,P18,P19,P2,P20,P21,P22,P23,P24,P25,P26,P27,P28,P29,P3,P30,P31,P32,P33,P34,P35,P36,P37,P4,P5,P6,P7,P8,P9,Type,isBig,isIL,revenue,missingSource
0,5647,İstanbul,Big Cities,0,07/17/1999,4,5,3,5,5.0,1,2,2,2,4,5,5.0,4,1,3,3,1,1,1.0,4.0,2.0,3.0,4.0,5,3,4,5,5,4,3,4,4.0,2,2,5,4,5,IL,True,True,5653753.000000,False
1,2513,Ankara,Big Cities,1,02/14/2008,4,5,1,5,5.0,0,0,0,0,0,3,5.0,2,1,3,2,0,0,0.0,0.0,3.0,3.0,4.0,0,0,0,0,0,0,0,0,4.0,1,2,5,5,5,FC,True,False,6923131.000000,True
2,663,Diyarbakır,Other,2,03/09/2013,2,5,2,5,5.0,0,0,0,0,0,1,4.0,1,1,1,1,0,0,0.0,0.0,1.0,3.0,2.0,0,0,0,0,0,0,0,0,5.0,2,3,5,5,5,IL,False,True,2055379.000000,True
3,1064,Tokat,Other,3,02/02/2012,6,10,8,10,7.5,6,4,9,3,12,20,4.5,12,6,1,10,2,2,2.5,2.5,2.5,7.5,6.0,25,12,10,6,18,12,12,6,6.0,4,4,10,8,10,IL,False,True,2675511.000000,False
4,2063,Gaziantep,Other,4,05/09/2009,3,5,2,5,5.0,2,1,2,1,4,2,4.0,2,1,2,1,2,3,3.0,5.0,1.0,3.0,3.0,5,1,3,2,3,4,3,3,4.0,2,2,5,5,5,IL,False,True,4316715.000000,False
5,1784,Ankara,Big Cities,5,02/12/2010,6,8,10,8,6.0,0,0,0,0,0,5,6.0,6,3,1,5,0,0,0.0,0.0,7.5,5.0,4.5,0,0,0,0,0,0,0,0,7.5,8,10,10,8,8,FC,True,False,5017319.000000,True
6,1543,İstanbul,Big Cities,6,10/11/2010,2,5,2,5,5.0,3,4,4,3,4,2,3.0,4,1,2,1,5,4,4.0,5.0,1.0,3.0,4.0,4,5,2,2,3,5,4,4,4.0,1,5,5,5,5,IL,True,True,5166635.000000,False
7,1290,İstanbul,Big Cities,7,06/21/2011,4,4,4,3,4.0,0,0,0,0,0,3,5.0,5,2,4,2,0,0,0.0,0.0,3.0,2.0,4.0,0,0,0,0,0,0,0,0,5.0,2,3,5,4,4,IL,True,True,4491607.000000,True
8,1587,Afyonkarahisar,Other,8,08/28/2010,1,5,1,5,5.0,1,1,2,1,4,1,1.0,1,1,1,1,4,4,4.0,2.0,2.0,3.0,4.0,4,5,5,3,4,5,4,5,4.0,1,2,1,5,5,IL,False,True,4952497.000000,False
9,1142,Edirne,Other,9,11/16/2011,6,10,2,10,7.5,0,0,0,0,0,25,4.5,3,3,1,10,0,0,0.0,0.0,5.0,2.5,6.0,0,0,0,0,0,0,0,0,7.5,6,4,10,10,10,IL,False,True,5444227.000000,True


In [79]:
misSource = data['missingSource'].values

In [89]:
n_samp = 250

for icol,col in enumerate(cols):
    X_ = X[:,[icol]]
    y_ = y
    
    if col in params.xor_cols:
        print('Reducing to valid entries')
        X_ = X_[np.logical_not(misSource),:]
        y_ = y[np.logical_not(misSource)]
    
    print(X_.shape)
    
    # Regular
    mu0,se0 = get_cv_results(X_, y_, n_samp=n_samp)
    print('{} def     {}+/-{}'.format(col,mu0,se0))
    
    # OneHot
    X_2 = sklearn.preprocessing.OneHotEncoder(sparse=False).fit_transform(X_)
    mu1,se1 = get_cv_results(X_2, y_, n_samp=n_samp)
    print('{} OneHot  {}+/-{}'.format(col,mu1,se1))
    
    # BoxCox
    X_2 = scalers.BoxCoxScaler().fit_transform(X_)
    mu2,se2 = get_cv_results(X_2, y_, n_samp=n_samp)
    print('{} BoxCox  {}+/-{}'.format(col,mu2,se2))
    
    # OrdinalHot
    X_2 = scalers.OrdinalHotEncoder().fit_transform(X_)
    mu3,se3 = get_cv_results(X_2, y_, n_samp=n_samp)
    print('{} Ordinal {}+/-{}'.format(col,mu3,se3))
    
    if abs(mu2 - mu1) < (se1 + se2):
        print('Results inconclusive for ' + col)
    elif mu1+se1 < mu2-se2 and mu1+se1 < mu3+se3:
        print('Better as a OneHot for ' + col)
    elif mu2+se2 < mu1-se1 and mu2+se2 < mu3-se3:
        print('Better as a BoxCox for ' + col)
    elif mu3+se3 < mu1-se1 and mu3+se3 < mu2-se2:
        print('Better as a OrdinalHot for ' + col)
    if mu0 < mu1 and mu0 < mu2 and mu0 < mu3:
        print('(Or better with the default)')
    
    print()

(166, 1)
P1 def     2514873.249940538+/-1072.2768603045756
P1 OneHot  2522699.3157757195+/-7650.72845965103
P1 BoxCox  2492078.1318384954+/-1033.451003456159
P1 Ordinal 2519059.2958056666+/-6885.309342073079
Better as a BoxCox for P1

(166, 1)
P2 def     2460670.568313692+/-1024.2193989279856
P2 OneHot  2394470.6259497493+/-3098.45914776928
P2 BoxCox  2459360.158127324+/-1009.2978986243658
P2 Ordinal 2399786.130758252+/-2954.8065042295398
Better as a OneHot for P2

(166, 1)
P3 def     2519912.527778374+/-1189.290825300454
P3 OneHot  1.4149531241104435e+18+/-2.865216808208465e+18
P3 BoxCox  2519791.9406722276+/-1166.3908301106346
P3 Ordinal 2.1337416001851907e+17+/-1.0647349204597775e+18
Results inconclusive for P3

(166, 1)
P4 def     2518697.12224351+/-1108.0909845801032
P4 OneHot  2551201.075728603+/-4343.623816339866
P4 BoxCox  2517331.909945769+/-1153.2661849740994
P4 Ordinal 1.1129517511376168e+18+/-2.7660242843414385e+18

(166, 1)
P5 def     2518579.6245857785+/-1200.467624207417